In [ ]:
!pip install -U transformers==3.0.0
!python -m nltk.downloader punkt
!git clone https://github.com/patil-suraj/question_generation.git

     |████████████████████████████████| 757kB 18.7MB/s 
     |████████████████████████████████| 3.0MB 53.6MB/s 
     |████████████████████████████████| 1.2MB 45.7MB/s 
     |████████████████████████████████| 890kB 47.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=45ab28e7e10c9c3ed83fe2e2bba49775ef223a81d67f17980469233a8e52f968
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
/usr/lib/python3.6/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Cloning into 'question_generation'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compress

In [ ]:
%cd question_generation

/content/question_generation


In [ ]:
from pipelines import pipeline
import json

In [ ]:
def getPassages(filename):
    with open(filename) as file:
        data = json.load(file)
    passages = list()
    for i in data["vertices"]["paragraphs"]:
      passages.append(i["text"])
    return passages

In [ ]:
def write_json(data, filename):
    with open(filename, 'w') as file:
        json.dump(data, file)

In [ ]:
def getTopics(filename, paras):
    with open(filename) as file:
        data = json.load(file)
    topics = ['' for i in range(len(data["vertices"]["topics"]))]
    edges = data["edges"]["main"]
    vertices = data["vertices"]["topics"]
    count = 0
    # print(edges)
    for e in edges:
        if e[1] == "about_concept":
          # print(e)
            topic_id = int(str(e[2])[2:])
            para_id = int(str(e[0])[2:])
            paras[para_id]=  vertices[topic_id]["text"] + "; " + paras[para_id]
            count += 1
    return paras

In [ ]:
def getTokenizedAnswers(context, answers):
  start_word = answers[:2]
  # print("context:",context,"answer:", answers)
  start_answer = context.index(answers)
  end_answer = start_answer + len(answers)
  return {
    "answer": answers,
    "start_answer": start_answer,
    "end_answer": end_answer
  }

In [ ]:
passages = getPassages("../handbook_graph.json")
passages = getTopics("../handbook_graph.json", passages)
print(passages)

['general ; This document gives the general regulations applicable to all btech programs. Specific requirements for a particular btech program (example btech in cse) are specified in regulations for program (example btech ', 'general ; regulations for btech programs ; While the Senate is the main statutory body for all academic matters, the Undergraduate Committee (UGC), a standing committee of Senate, shall oversee matters related to the undergraduate programs. the Undergraduate Committee (UGC), a standing committee of Senate shall be appointed by the Senate and shall have a term of two years. the Undergraduate Committee (UGC), a standing committee of Senate will consist of Faculty members and members of the Senate. In addition, there will be at least one student representative, who will be a full time UG student of the Institute. ', 'regulations for btech programs ; general ; Any condition arising in the btech program and not covered in the regulations shall be referred to the UG com

In [ ]:
with open("../preprocessed_handbook.json") as file:
  data = json.load(file)
for id in data["paras"]:
  para = data["paras"][id]
  topic = data["topics"][str(data["edges"][id]["topic"])]
  passages.append(topic+"; "+para)

In [ ]:
qg = pipeline("question-generation", model="valhalla/t5-base-qg-hl")

In [ ]:
qaList = list()
count = 0
questions_dictionary = dict()
id = 0
countQuestions = 0
for p in passages:
  try:
    qaList = qg(p)
    questions_dictionary[id] = {
        'qas':dict(),
        'context':p
    }
    qid = 0
    qa = questions_dictionary[id]["qas"]
    for questions in qaList:
      countQuestions += 1
      qa[qid] = dict()
      answers = getTokenizedAnswers(p, questions["answer"])
      qa[qid]["question"] = questions["question"]
      qa[qid]["answer"] = answers
      qid += 1
    id += 1
  except:
    count += 1
    print(p)
    print('error')
    continue
print(countQuestions)

continuing in the program ; academic load ; Students can avail underload below 12 credits (as low as 4 credits) only in one semester during Students entire btech program. For this, prior permission is required to be taken before the last date of add or drop of that particular semester. Students wanting to avail internship (industrial, academic, research, etc.) without taking semester leave can do so with prior approval of the DOAA in only one semester of Students wanting to avail internship (industrial, academic, research, etc.) without taking semester leave degree program. Such request needs to be placed at least two days before the last date of add or drop. Students approved for such internship are required to register for 4 credits of only IP or IS or UR or BTP or OC courses in this semester. Students will not be allowed to do any in-class course during the internship period. Further, the underload of as low as 4 credits can be availed by ECE, CSAM and CSSS students in any semester 

In [ ]:
write_json(questions_dictionary, "training_set_nueral.json")